#####################################
#                                   #
#           Import                  #
#                                   #
#                                   #
#####################################

Importing packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.cluster.hierarchy as sch
import scipy.stats as stats
from scipy.stats import zscore
from scipy.spatial.distance import pdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import pingouin as pg
import plotly.express as px 
import plotly.io as pio
pio.renderers.default='browser'
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
import sympy as sy
from scipy.stats import chi2_contingency
import statsmodels.api as sm
import plotly.graph_objects as go
import prince


Importing Database

In [ ]:
db_pokemon = pd.read_csv('pokemon.csv')

We want to select only the 1st generation = keeping it old school!

In [ ]:
db_pokemon_1stgen = db_pokemon[db_pokemon['generation'] == 1]
print(db_pokemon_1stgen.info())
descritive_db_pokemon = db_pokemon_1stgen.describe()

3d chart of observations based on initial stats = 'HP', 'Attack', 'Defense'.
One of the goals of this analysis is to have the same 3d chart but with clustered Pokémon.

In [ ]:
fig = px.scatter_3d(db_pokemon_1stgen, 
                    x='hp', 
                    y='attack', 
                    z='defense',
                    text=db_pokemon_1stgen.name)
fig.show()

#####################################
#                                   #
#          PCA                      #
#                                   #
#                                   #
#####################################

Goal: Identify underlying relationships between variables by grouping them into factors.

Variables: initial status of Pokémon (HP, Attack, Defense, SP_Attack, SP_Defense).

To start the factor analysis, let's start by selecting variables = only attribute stats.

In [ ]:
db_pokemon_1stgen_stats = db_pokemon_1stgen[['hp','attack','defense','sp_attack','sp_defense']]

Then we move on with Pearson's correlation of the variables.

With this approach we can see how statistically related the variables are.

The upper triangle of the matrix shows the significance level, 
which is the probabilty of observing the correlation coefficient under the null hypothesis -that there is no correlation between the pair of variables-.

In [ ]:
pearson_correlation_stats = pg.rcorr(db_pokemon_1stgen_stats, method = 'pearson', upper = 'pval', 
         decimals = 4, 
         pval_stars = {0.01: '***', 0.05: '**', 0.10: '*'})

Let's also calculate Bartlett's test of sphericity

In [ ]:
bartlett, p_value = calculate_bartlett_sphericity(db_pokemon_1stgen_stats)

print(f'Qui² Bartlett: {round(bartlett, 2)}')
print(f'p-value: {round(p_value, 4)}')

Based on p-value we can move on with using Factor analysis.

The next step is to define PCA (1st with all available factors)

In [ ]:
fa = FactorAnalyzer(n_factors=5, method='principal', rotation=None).fit(db_pokemon_1stgen_stats)

Then we should be obtaining Eigenvalues, variances and cumulative variances 

In [ ]:
eigenvalues_factors= fa.get_factor_variance()

table_eigen = pd.DataFrame(eigenvalues_factors)
table_eigen.columns = [f"Factor {i+1}" for i, v in enumerate(table_eigen.columns)]
table_eigen.index = ['Eigenvalue','Variance', 'Cumulative Variance']
table_eigen = table_eigen.T

print(table_eigen)

It's possible to see that 3 factors result in 83% of the cumulative variance.

Let's move on by obtaining factorial loadings. 
With that we can understand what is the importance of each variable when the factor is being defined.

In [ ]:
factorial_loadings = fa.loadings_

table_loads = pd.DataFrame(factorial_loadings)
table_loads.columns = [f"Factor {i+1}" for i, v in enumerate(table_loads.columns)]
table_loads.index = db_pokemon_1stgen_stats.columns

print(table_loads)

Let's analyze that visually...

In [ ]:
#%% Analyzing factorial loads of each factor 

table_loads_graph = table_loads.reset_index()
table_loads_graph = table_loads_graph.melt(id_vars='index')

sns.barplot(data=table_loads_graph, x='variable', y='value', hue='index', palette='bright')
plt.legend(title='Variables', bbox_to_anchor=(1,1), fontsize = '6')
plt.title('Factorial loads', fontsize='12')
plt.xlabel(xlabel=None)
plt.ylabel(ylabel=None)
plt.show()

It's possible to see that 3 factors result in 83% of the cumulative variance. We'll use only those 3 factors as the variables for the clustering of Pokémon.

In [ ]:
fa = FactorAnalyzer(n_factors=3, method='principal', rotation=None).fit(db_pokemon_1stgen_stats)

Obtaining Eigenvalues, variances and cumulative variances for new PCA.

In [ ]:
eigenvalues_factors= fa.get_factor_variance()

table_eigen = pd.DataFrame(eigenvalues_factors)
table_eigen.columns = [f"Factor {i+1}" for i, v in enumerate(table_eigen.columns)]
table_eigen.index = ['Eigenvalue','Variance', 'Cumulative Variance']
table_eigen = table_eigen.T

print(table_eigen)

# It's possible to see that 3 factors result in 83% of the cumulative variance.


Extracting factors to the observations on the database

In [ ]:
factors = pd.DataFrame(fa.transform(db_pokemon_1stgen_stats))
factors.columns =  [f"Factor {i+1}" for i, v in enumerate(factors.columns)]

Adding factors to the database

In [ ]:
db_pokemon_1stgen = pd.concat([db_pokemon_1stgen.reset_index(drop=True),factors], axis=1)

With that, we conclude that 5 different variables were grouped into 3 different factors that sum up to 83% of the cumulative variance.

#####################################
#                                   #
#              Ranking              #
#                                   #
#                                   #
#####################################

Let's rank the Pokémon based on the factors

In [ ]:
db_pokemon_1stgen['Ranking'] = 0

for index, item in enumerate(list(table_eigen.index)):
    variance = table_eigen.loc[item]['Variance']

    db_pokemon_1stgen['Ranking'] = db_pokemon_1stgen['Ranking'] + db_pokemon_1stgen[table_eigen.index[index]]*variance

A Ranking column was created so we can say which are the best pokemons based on their Factor results.

#####################################
#                                   #
#            Clustering             #
#                                   #
#                                   #
#####################################

Goal: Segment Pokémon with similar initial stats into groups.

Variables: Factors from Factor Analysis.

Selecting variables for clustering


In [ ]:
db_pokemon_1stgen_factor_stats = db_pokemon_1stgen[['Factor 1','Factor 2','Factor 3']]

It's up to us to decide how many clusters we want data segmented into, but there are some techniques that might help us on that. One of them is the Elbow technique.

Elbow - a support method as a suggestion for the quantity of clusters

In [ ]:
elbow = []
K = range(1,40) # stop point should be a manual input 
for k in K:
    kmeanElbow = KMeans(n_clusters=k, init='random', random_state=100).fit(db_pokemon_1stgen_factor_stats)
    elbow.append(kmeanElbow.inertia_)
    
plt.figure(figsize=(16,8))
plt.plot(K, elbow, marker='o')
plt.xlabel('# of Clusters', fontsize=16)
plt.xticks(range(1,40))
plt.ylabel('WCSS', fontsize=16)
plt.title('Elbow method', fontsize=16)
plt.show()

According to the analysis we decided to keep 5 clusters.

So let's cluster Pokémon by the method K-means.

In [ ]:
kmeans = KMeans(n_clusters=5, init='random', random_state=100).fit(db_pokemon_1stgen_factor_stats)

kmeans_clusters = kmeans.labels_
db_pokemon_1stgen['cluster_kmeans'] = kmeans_clusters
db_pokemon_1stgen['cluster_kmeans'] = db_pokemon_1stgen['cluster_kmeans'].astype('category')

Then let's plot the clusters using the k-means method

In [ ]:
fig_clustering_kmeans = px.scatter_3d(db_pokemon_1stgen, 
                    x='Factor 1', 
                    y='Factor 2', 
                    z='Factor 3',
                    color='cluster_kmeans',
                    text=db_pokemon_1stgen.name)
fig_clustering_kmeans.show()


We now have 5 different groups of Pokémon based on their innitial stats.

#####################################
#                                   #
#               CA                  #
#                                   #
#                                   #
#####################################

The Simple Correspondence Analysis will be used so we can understand if there is any relationship between the cluster and the ranking. 

Goal: Understand if Pokémon of a specific group are better or worse when compared to other groups.

Goal2: What's the best cluster based on their ranking positions.

Variables: initial status of Pokémon (Groups based on ranks, Clusters based on initial stats).

We should start by categorizing the Ranking column into 5 equal groups.
It's ascending so let's label the groups accordingly - from worst to best -

In [ ]:
db_pokemon_1stgen['Ranking_category'] = pd.qcut(db_pokemon_1stgen['Ranking'],5,labels=['Worst_Group','Bad_Group','Medium_group','Good_group','Best_group'])

Is there any association between cluster and ranking group?

With crosstab we can count the number of occurences on a pair of characteristics, as it displays the frequency distribution of the variables.

In [ ]:
comparison_table = pd.crosstab(db_pokemon_1stgen['cluster_kmeans'],db_pokemon_1stgen['Ranking_category'])
print(comparison_table)

qui2_test = chi2_contingency(comparison_table)

print(f"qui² statistic: {round(qui2_test[0], 2)}")
print(f"p-value: {round(qui2_test[1], 4)}")
print(f"Degrees of Freedom: {qui2_test[2]}")


Analyzing standardized resids, which are the differences between the observed values and the expected values for each pair of occurences.

In [ ]:
tab_res = sm.stats.Table(comparison_table)
print(tab_res.standardized_resids)

fig1 = go.Figure()

maxz = np.max(tab_res.standardized_resids)+0.1
minz = np.min(tab_res.standardized_resids)-0.1

colorscale = ['purple' if i>1.96 else '#FAF9F6' for i in np.arange(minz,maxz,0.01)]

fig1.add_trace(
    go.Heatmap(
        x = tab_res.standardized_resids.columns,
        y = tab_res.standardized_resids.index,
        z = np.array(tab_res.standardized_resids),
        text=tab_res.standardized_resids.values,
        texttemplate='%{text:.2f}',
        showscale=False,
        colorscale=colorscale))

fig1.update_layout(
    title='<b>Standardized adjusted resids</b>',
    height = 600,
    width = 600)

fig1.show()

Correspondence Analysis

In [ ]:
ca = prince.CA().fit(comparison_table)

Obtaining Eigenvalues for CA

In [ ]:
eigen_table = ca.eigenvalues_summary

print(eigen_table)

Plotting the CA map

In [ ]:
chart_df_row = pd.DataFrame({'var_row': comparison_table.index,
                             'x_row':ca.row_coordinates(comparison_table)[0].values,
                             'y_row': ca.row_coordinates(comparison_table)[1].values})

chart_df_col = pd.DataFrame({'var_col': comparison_table.columns,
                             'x_col':ca.column_coordinates(comparison_table)[0].values,
                             'y_col': ca.column_coordinates(comparison_table)[1].values})

def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'] + 0.03, point['y'] - 0.02, point['val'], fontsize=6)

label_point(x = chart_df_col['x_col'],
            y = chart_df_col['y_col'],
            val = chart_df_col['var_col'],
            ax = plt.gca())

label_point(x = chart_df_row['x_row'],
            y = chart_df_row['y_row'],
            val = chart_df_row['var_row'],
            ax = plt.gca()) 

sns.scatterplot(data=chart_df_row, x='x_row', y='y_row', s=20)
sns.scatterplot(data=chart_df_col, x='x_col', y='y_col', s=20)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.axhline(y=0, color='lightgrey', ls='--')
plt.axvline(x=0, color='lightgrey', ls='--')
plt.tick_params(size=2, labelsize=6)
plt.title("Mapa Perceptual - Anacor", fontsize=12)
plt.xlabel(f"Dim. 1: {eigen_table.iloc[0,1]} da inércia", fontsize=8)
plt.ylabel(f"Dim. 2: {eigen_table.iloc[1,1]} da inércia", fontsize=8)
plt.show()

#####################################
#                                   #
#              Conclusion           #
#                                   #
#                                   #
#####################################

It's easy to see that:

Clusters 1 and 2
Majorly groups the Pokémon with the worst charactheristics.

Cluster 3
Majorly groups the Pokémon with good charactheristics.


Analyzing the CA Map and the standardized resids one can affirm that:

Cluster 0
Majorly groups the Pokémon with medium charactheristics.

and finally

Cluster 4
Majorly groups the Pokémon with the best charachteristics.




Let's print the names and status of Pokémon of Cluster 4.

In [ ]:
Cluster_4_pokemons = db_pokemon_1stgen[db_pokemon_1stgen['cluster_kmeans'] == 4][['name','hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'Ranking']].sort_values(by='Ranking',ascending=False)
print(Cluster_4_pokemons.describe().round(2))

#####################################
#                                   #
#              Exporting            #
#                                   #
#                                   #
#####################################

Exporting analysis

In [ ]:
filename = 'Pokemon-Unsupervised ML analysis.csv'
db_pokemon_1stgen.to_csv(filename, index=False,sep =";")

#####################################
#                                   #
#              Appendix             #
#                                   #
#                                   #
#####################################

Alternative Method for clustering  = Agglomerative Hierarchical Cluster: Euclidian + single linkage

In [ ]:
euclidian = pdist(db_pokemon_1stgen_factor_stats, metric='euclidean')

1st: let's plot Dendogram for single method, and compare it to other dendograms

In [ ]:
plt.figure(figsize=(16,8))
dend_sing = sch.linkage(db_pokemon_1stgen_factor_stats, method = 'single', metric = 'euclidean')
dendrogram_s = sch.dendrogram(dend_sing, color_threshold = 4.5, labels = list(db_pokemon_1stgen.name))
plt.title('Dendrograma', fontsize=16)
plt.xlabel('Pokemon', fontsize=16)
plt.ylabel('Distância Euclidiana', fontsize=16)
plt.show()

2nd: Dendogram for complete method

In [ ]:
plt.figure(figsize=(16,8))
dend_comp = sch.linkage(db_pokemon_1stgen_factor_stats, method = 'complete', metric = 'euclidean')
dendrogram_s = sch.dendrogram(dend_comp, color_threshold = 4.5, labels = list(db_pokemon_1stgen.name))
plt.title('Dendrograma', fontsize=16)
plt.xlabel('Pokemon', fontsize=16)
plt.ylabel('Distância Euclidiana', fontsize=16)
plt.show()

3rd: Dendogram for average method

In [ ]:
plt.figure(figsize=(16,8))
dend_avg = sch.linkage(db_pokemon_1stgen_factor_stats, method = 'average', metric = 'euclidean')
dendrogram_s = sch.dendrogram(dend_avg, color_threshold = 4.5, labels = list(db_pokemon_1stgen.name))
plt.title('Dendrograma', fontsize=16)
plt.xlabel('Pokemon', fontsize=16)
plt.ylabel('Distância Euclidiana', fontsize=16)
plt.show()

In [ ]:
cluster_complete = AgglomerativeClustering(n_clusters = 5, metric = 'euclidean', linkage = 'complete')
indica_cluster_complete = cluster_complete.fit_predict(db_pokemon_1stgen_factor_stats)
db_pokemon_1stgen['cluster_complete'] = indica_cluster_complete
db_pokemon_1stgen['cluster_complete'] = db_pokemon_1stgen['cluster_complete'].astype('category')

Plotting the clusters using the Agglomerative Clustering method

In [ ]:
fig_clustering_agglomerative = px.scatter_3d(db_pokemon_1stgen, 
                    x='Factor 1', 
                    y='Factor 2', 
                    z='Factor 3',
                    color='cluster_complete',
                    text=db_pokemon_1stgen.name)
fig_clustering_agglomerative.show()
